In [1]:
! rclone listremotes

denali:
tallgrass:
wss3:


In [2]:
#! rclone size wss3:/ws-in/

In [3]:
#! mkdir s3lib

In [4]:
#!wget https://raw.githubusercontent.com/tonybutzer/data-curation/79323ea534ccce76bd8026454a013fb436395c6a/ips-study/lib/ipslib/ipslib/bucket_analyze.py

In [5]:
#! mv bucket_analyze.py s3lib

In [6]:
import sys

sys.path.append('./s3lib')

In [7]:
from bucket_analyze import bucket_analyze

In [8]:
! rclone help flags | grep depth

      --max-depth int                        If set limits the recursion depth to this. (default -1)


In [9]:
! rclone lsd wss3:/ws-in --max-depth 1

           0 2022-01-08 21:24:55        -1 CONUS
           0 2022-01-08 21:24:55        -1 DelawareRiverBasin
           0 2022-01-08 21:24:55        -1 EastAfrica
           0 2022-01-08 21:24:55        -1 Marshall_Islands
           0 2022-01-08 21:24:55        -1 NorthAmerica
           0 2022-01-08 21:24:55        -1 WOTJE-code
           0 2022-01-08 21:24:55        -1 cfactor
           0 2022-01-08 21:24:55        -1 global_dT
           0 2022-01-08 21:24:55        -1 home
           0 2022-01-08 21:24:55        -1 tony


In [10]:
def get_bucket_list():
    '''
    ___________________________________________________________________
    THIS function returns watersmart S3 bucket names
    __________________________________________________________________
      - we only need THREE (3) buckets to do our work
    our buckets are used for our data inputs
          - ws-in: holds precip, NDVI, Temperature - inputs to our water models
          - ws-out: is our model outputs - sometimes just tiles and chips
          - ws-enduser: - is where we mosaic these into usable tifs, netcdfs, ugly-tarballs 
          ... etc ...
    '''
    b_list = ['ws-in', 'ws-out', 'ws-enduser']
    return b_list
    

In [11]:
bucket_list = get_bucket_list()

In [12]:
import boto3
def s3_list_pseudo_subdirs(bucket, prefix_with_slash):

    subfolder_list = []
    #Make sure you provide / in the end

    a = prefix_with_slash.split('/')
    prefix_with_slash='/'.join(a[1:])
    prefix = prefix_with_slash 
    
    print('prefix_with_slash', prefix_with_slash)

    client = boto3.client('s3')
    result = client.list_objects(Bucket=bucket, Prefix=prefix, Delimiter='/')
    for o in result.get('CommonPrefixes'):
        #print ('sub folder : ', o.get('Prefix'))
        subfolder_list.append(o.get('Prefix'))
    return subfolder_list


In [13]:
bucket='ws-in'
prefix_with_slash = '/'  # top dirs
my_subdir_list=s3_list_pseudo_subdirs(bucket, prefix_with_slash)


prefix_with_slash 


In [14]:
my_subdir_list

['CONUS/',
 'DelawareRiverBasin/',
 'EastAfrica/',
 'Marshall_Islands/',
 'NorthAmerica/',
 'WOTJE-code/',
 'cfactor/',
 'global_dT/',
 'home/',
 'tony/']

In [15]:
du_list_of_dicts = []
for bucket_name in bucket_list:
    prefix_with_slash = '/'  # top dirs
    my_subdir_list=s3_list_pseudo_subdirs(bucket_name, prefix_with_slash)
    for pdir in my_subdir_list:
        gb, dol_month = bucket_analyze(bucket_name, pdir)
        du_dict = {
            'bucket':bucket_name,
            'DirPrefix':pdir,
            'Gbytes':gb,
            'Costs':dol_month
        }
        du_list_of_dicts.append(du_dict)

prefix_with_slash 
bucket ws-in
prefix CONUS/
COUNT= 34092
STANDARD 34092
STANDARD 1237631520012
STANDARD GBYTES= 1152.634173642844
STANDARD Cost/Month= 26.510585993785412
----------------------------------------------------------------------------------------------------
GLACIER 0
GLACIER 0
GLACIER GBYTES= 0.0
GLACIER Cost/Month= 0.0
----------------------------------------------------------------------------------------------------
INTELLIGENT_TIERING 0
INTELLIGENT_TIERING 0
INTELLIGENT_TIERING GBYTES= 0.0
INTELLIGENT_TIERING Cost/Month= 0.0
----------------------------------------------------------------------------------------------------
END LOOP
G: 1152.634173642844 26.510585993785412
bucket ws-in
prefix DelawareRiverBasin/
COUNT= 795894
STANDARD 795894
STANDARD 749830659763
STANDARD GBYTES= 698.3342205761
STANDARD Cost/Month= 16.0616870732503
----------------------------------------------------------------------------------------------------
GLACIER 0
GLACIER 0
GLACIER GBYTES= 0

COUNT= 206248
STANDARD 206248
STANDARD 3834375615517
STANDARD GBYTES= 3571.0405702861026
STANDARD Cost/Month= 82.13393311658035
----------------------------------------------------------------------------------------------------
GLACIER 0
GLACIER 0
GLACIER GBYTES= 0.0
GLACIER Cost/Month= 0.0
----------------------------------------------------------------------------------------------------
INTELLIGENT_TIERING 0
INTELLIGENT_TIERING 0
INTELLIGENT_TIERING GBYTES= 0.0
INTELLIGENT_TIERING Cost/Month= 0.0
----------------------------------------------------------------------------------------------------
END LOOP
G: 3571.0405702861026 82.13393311658035
bucket ws-out
prefix good/
COUNT= 39
STANDARD 39
STANDARD 24382678253
STANDARD GBYTES= 22.708138686604798
STANDARD Cost/Month= 0.5222871897919104
----------------------------------------------------------------------------------------------------
GLACIER 0
GLACIER 0
GLACIER GBYTES= 0.0
GLACIER Cost/Month= 0.0
---------------------------------

In [16]:
du_list_of_dicts

[{'bucket': 'ws-in',
  'DirPrefix': 'CONUS/',
  'Gbytes': 1152.634173642844,
  'Costs': 26.510585993785412},
 {'bucket': 'ws-in',
  'DirPrefix': 'DelawareRiverBasin/',
  'Gbytes': 698.3342205761,
  'Costs': 16.0616870732503},
 {'bucket': 'ws-in',
  'DirPrefix': 'EastAfrica/',
  'Gbytes': 25.73511326033622,
  'Costs': 0.5919076049877331},
 {'bucket': 'ws-in',
  'DirPrefix': 'Marshall_Islands/',
  'Gbytes': 3.0795034440234303,
  'Costs': 0.07082857921253889},
 {'bucket': 'ws-in',
  'DirPrefix': 'NorthAmerica/',
  'Gbytes': 2360.6205078894272,
  'Costs': 54.29427168145683},
 {'bucket': 'ws-in', 'DirPrefix': 'WOTJE-code/', 'Gbytes': 1, 'Costs': 0.023},
 {'bucket': 'ws-in',
  'DirPrefix': 'cfactor/',
  'Gbytes': 149.86372096277773,
  'Costs': 3.4468655821438876},
 {'bucket': 'ws-in',
  'DirPrefix': 'global_dT/',
  'Gbytes': 4.07246426679194,
  'Costs': 0.09366667813621461},
 {'bucket': 'ws-in',
  'DirPrefix': 'home/',
  'Gbytes': 9.139479291625321,
  'Costs': 0.21020802370738237},
 {'bucket

In [17]:
import pandas as pd

In [18]:
df = pd.DataFrame(du_list_of_dicts)

In [19]:
df

,bucket,DirPrefix,Gbytes,Costs
0,ws-in,CONUS/,1152.634174,26.510586
1,ws-in,DelawareRiverBasin/,698.334221,16.061687
2,ws-in,EastAfrica/,25.735113,0.591908
3,ws-in,Marshall_Islands/,3.079503,0.070829
4,ws-in,NorthAmerica/,2360.620508,54.294272
5,ws-in,WOTJE-code/,1.000000,0.023000
6,ws-in,cfactor/,149.863721,3.446866
7,ws-in,global_dT/,4.072464,0.093667
8,ws-in,home/,9.139479,0.210208
9,ws-in,tony/,1.000000,0.023000


In [23]:
mdf = df.loc[df['Gbytes'] > 1]

In [21]:
#!rclone size wss3:/ws-enduser/DelawareRiverBasin

In [22]:
#!rclone size wss3:/ws-in/DelawareRiverBasin

In [24]:
mdf

,bucket,DirPrefix,Gbytes,Costs
0,ws-in,CONUS/,1152.634174,26.510586
1,ws-in,DelawareRiverBasin/,698.334221,16.061687
2,ws-in,EastAfrica/,25.735113,0.591908
3,ws-in,Marshall_Islands/,3.079503,0.070829
4,ws-in,NorthAmerica/,2360.620508,54.294272
6,ws-in,cfactor/,149.863721,3.446866
7,ws-in,global_dT/,4.072464,0.093667
8,ws-in,home/,9.139479,0.210208
10,ws-out,DelawareRiverBasin/,22704.247166,522.197685
11,ws-out,USA/,24050.205945,553.154737


In [25]:
mdf.to_csv('./ws_s3_usage.csv', index=False)

In [26]:
! ls *.csv

ws_s3_usage.csv
